In [518]:
google_key=?????????

SyntaxError: invalid syntax (<ipython-input-518-f7608a99e254>, line 1)

# What we want?
- house
- \>3rooms (+livingroom +kitchen)
- garden
- parking
- <€1000
- cheapest

Additional (for enrichments):
- closest to University of Twente (5, Drienerlolaan, 7522 NB Enschede)
- best energy label 

### Source of houses
- https://www.funda.nl/
- Ex1: https://www.funda.nl/huur/enschede/+5km/
- Ex2: https://www.funda.nl/huur/enschede/woonhuis/3+kamers/tuin/+5km/
- Ex3: https://www.funda.nl/huur/enschede/0-1000/woonhuis/3+kamers/tuin/+5km/

### Other source of information

- https://www.huurwoningen.nl
- https://www.pararius.nl
- https://www.huislijn.nl 
- https://directwonen.nl

### Energy label (energielabel)
- https://www.energielabel.nl/woningen/zoek-je-energielabel/

In [475]:
import cfscrape
from lxml import etree
import pandas as pd
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
    
def get_links_funda_query(link):
    all_links=[]
    page=1
    raw_link = link
    
    while True:
        print("Scraping page",page)
        scraper = cfscrape.create_scraper()    
        scraped_html=scraper.get(link).content.decode("utf-8") 
        parsed_html = BeautifulSoup(scraped_html)

        data = parsed_html.body.find_all('div', attrs={'class':'search-result-media'})
       
        if data:
            for div in data:
                links = div.findAll('a')
                for a in links:
                    all_links.append("https://www.funda.nl" + a['href'])

            if len(links)<1:
                break
            page=page+1
            link = raw_link+'p'+str(page)
        else:
            print("There is NO page",page,"!")
            break

    return all_links

In [477]:
links = get_links_funda_query("https://www.funda.nl/huur/hengelo/0-1000/woonhuis/3+kamers/tuin/+15km/")

# https://www.funda.nl/huur/hengelo/0-1000/woonhuis/3+kamers/tuin/+15km/

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
There is NO page 4 !


In [486]:
def scrape_funda_single_page (url):
    ###
    # Getting the content of a single post about a house
    scraper = cfscrape.create_scraper()
    try:
        scraped_html=scraper.get(url).content.decode("utf-8") 
        parsed_html = BeautifulSoup(scraped_html)
        
        ###
        # Initiating lists 
        ids = []
        attributes = []

        ###
        #Getting from the title
        description = parsed_html.find('title').text.replace('Huis te huur: ','').replace(' [funda]','')
        postal_code = re.search(r'\d{4} [A-Z]{2}', description).group(0)
        city = re.search(r'\d{4} [A-Z]{2} \w+',description).group(0).split()[2]
        address = re.findall(r'(.*) \d{4}',description)[0]

        ###
        #Adding content to lists
        ids.extend(['description','link','postal_code','city','address'])
        attributes.extend([description,url,postal_code, city, address])

        ###
        #Getting the specific content inside the body of the page
        data = parsed_html.body.find_all('div', attrs={'class':'object-kenmerken-body'})[0]

        ###
        #Getting <dt>
        for i in data.find_all('dt'):
            ids.append(i.text.splitlines()[0])

        ###
        #Getting <dd>
        for i in data.find_all('dd'):
            attributes.append(i.text.splitlines()[0])

        attributes.remove('')
        attributes.remove('Woonoppervlakte')

        ###
        #Mergind ids and attributes in a dataframe
        df_temp = pd.DataFrame()
        df_temp['Id'] = ids
        df_temp['Attribute'] = attributes

        ###
        #Transposing the dataframe
        df_temp_transposed = df_temp.transpose().rename(columns=df_temp['Id']).drop(['Id']).reset_index(drop=True)
        df_temp_transposed = df_temp_transposed.loc[:,~df_temp_transposed.columns.duplicated()]
        
        return df_temp_transposed

    except:
        next
   

In [487]:
###
#Concatenating all dataframes
df = pd.DataFrame()
for link in links:
    df_temp = scrape_funda_single_page(link)
    df = pd.concat([df,df_temp], axis=0, ignore_index=True)

# Bike Distance from UT

In [510]:
import googlemaps
from datetime import datetime

def gmaps_bike_distance (key, point_a, point_b):
    gmaps = googlemaps.Client(key=key)
    directions_result = gmaps.directions(point_a, point_b, mode="bicycling")
    bike_distance = directions_result[0][u'legs'][0][u'distance'][u'text'].replace(' km','')
    return float(bike_distance)

def bike_distance_from_ut(point_b):
    return gmaps_bike_distance (google_key, 'University of Twente', point_b)

## Enriching the distance from the house to UT 

In [507]:
df['distance_ut'] = df['address'].map(lambda x: bike_distance_from_ut(x))

## Displaying specific information

In [508]:
df[['link','distance_ut', 'Huurprijs ','Bouwjaar', 'Aantal kamers','Tuin']].sort_values(by=['distance_ut','Huurprijs '])

,link,distance_ut,Huurprijs,Bouwjaar,Aantal kamers,Tuin
12,https://www.funda.nl/huur/enschede/huis-40526049-hengelosestraat-138/,2.7,€ 995 per maand (geen servicekosten),1910,4 kamers (3 slaapkamers),Achtertuin
34,https://www.funda.nl/huur/hengelo/huis-40883295-hermesstraat-40/,4.2,€ 951 per maand (exclusief servicekosten à € 30 /mnd),Na 2011,5 kamers (3 slaapkamers),Achtertuin
15,https://www.funda.nl/huur/enschede/huis-40854757-ankrot-110/,4.2,NaN,NaN,6 kamers (3 slaapkamers),Achtertuin
29,https://www.funda.nl/huur/enschede/huis-40726524-roombeekstraat-60/,4.5,€ 975 per maand (exclusief servicekosten à € 20 /mnd),NaN,4 kamers (3 slaapkamers),Achtertuin en voortuin
35,https://www.funda.nl/huur/enschede/huis-40884037-laaressingel-103/,4.9,€ 783 per maand (geen servicekosten),1928,4 kamers (3 slaapkamers),Achtertuin
3,https://www.funda.nl/huur/enschede/huis-40809158-sniedershorst-68/,5.8,€ 985 per maand (geen servicekosten),NaN,6 kamers (5 slaapkamers),Achtertuin en zijtuin
27,https://www.funda.nl/huur/enschede/huis-40870907-vrugginklanden-12/,7.7,€ 908 per maand (geen servicekosten),NaN,4 kamers,Achtertuin en voortuin
11,https://www.funda.nl/huur/hengelo/huis-40681224-tom-mandersstraat-3/,7.8,€ 830 per maand (geen servicekosten),1982,5 kamers (4 slaapkamers),Achtertuin en voortuin
21,https://www.funda.nl/huur/enschede/huis-40870057-kalkerlanden-24/,7.9,€ 820 per maand (geen servicekosten),NaN,5 kamers (4 slaapkamers),Achtertuin en voortuin
0,https://www.funda.nl/huur/hengelo/huis-40898395-hammerfeststraat-25/,9.4,€ 850 per maand (geen servicekosten),1994,6 kamers (4 slaapkamers),Achtertuin en voortuin


# TODO:
- pre-process the content of some columns
- write an automated script for automaticaly running and notifying whether there is something new.
